In [2]:
import pandas as pd
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import cross_validate,KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
import sys
import csv

In [3]:
users=20
items=40
req_sim=0

In [37]:
data = readingFile('aman_rating.csv')
Mat = np.zeros((users,items))
for e in data:
    Mat[e[0]-1][e[1]-1] = e[2]
for i in range(users):
    print(Mat[i])

[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 

In [14]:
def readingFile(filename):
    f = open(filename,"r")
    data=[]
    ct=0
    for row in f:
        if ct==0:
            ct=1
            continue
        else:
            r = row.split(',')
            r[2] = r[2][:-2]
            e = [ int(r[0].strip(' " ')) , int(r[1]) , int(r[2]) ]
            data.append(e)
    f.close()
    return data

In [15]:
def similarity_user(data):
    user_similarity_cosine = np.zeros((users,users))
    user_similarity_jaccard = np.zeros((users,users))
    user_similarity_pearson = np.zeros((users,users))
    for user1 in range(users):
        for user2 in range(users):
            if np.count_nonzero(data[user1]) and np.count_nonzero(data[user2]):
                user_similarity_cosine[user1][user2] = 1 - scipy.spatial.distance.cosine(data[user1],data[user2])
                user_similarity_jaccard[user1][user2] = 1 - scipy.spatial.distance.jaccard(data[user1],data[user2])
                try:
                    if not math.isnan(scipy.stats.pearson(data[user1],data[user2])[0]):
                        user_similarity_pearson[user1][user2] = scipy.stats.pearson(data[user1],data[user2])[0]
                    else:
                        user_similarity_pearson[user1][user2] = 0
                except:
                    user_similarity_pearson[user1][user2] = 0
    return user_similarity_cosine , user_similarity_jaccard , user_similarity_pearson

In [16]:
def crossValidation(data):
    k_fold = KFold(n_splits=3,shuffle=True)
    Mat = np.zeros((users,items))
    for e in data:
        Mat[e[0]-1][e[1]-1] = e[2]

    sim_user_cosine , sim_user_jaccard , sim_user_pearson = similarity_user(Mat)

    rmse_cosine = []
    rmse_jaccard = []
    rmse_pearson = []

    for train_indices , test_indices in k_fold.split(data):
        train = [data[i] for i in train_indices]
        test = [data[i] for i in test_indices]
        M = np.zeros((users,items))
        for e in train:
            M[e[0]-1][e[1]-1] = e[2]

        true_rate = []
        pred_rate_cosine=[]
        pred_rate_jaccard=[]
        pred_rate_pearson=[]

        for e in test:
            user = e[0]
            item = e[1]
            true_rate.append(e[2])

            pred_cosine = 3.0
            pred_jaccard = 3.0
            pred_pearson = 3.0

            if np.count_nonzero(M[user-1]):
                sim_cosine = sim_user_cosine[user-1]
                sim_jaccard = sim_user_jaccard[user-1]
                sim_pearson = sim_user_pearson[user-1]
                ind = ( M[:,item-1] > 0 )
                normal_cosine = np.sum(np.absolute(sim_cosine[ind]))
                normal_jaccard = np.sum(np.absolute(sim_jaccard[ind]))
                normal_pearson = np.sum(np.absolute(sim_pearson[ind]))
                if normal_cosine > 0:
                    pred_cosine = np.dot(sim_cosine , M[:,item-1])/normal_cosine
                if normal_jaccard > 0:
                    pred_jaccard = np.dot(sim_jaccard , M[:,item-1])/normal_jaccard
                if normal_pearson > 0:
                    pred_pearson = np.dot(sim_pearson , M[:,item-1])/normal_pearson

            if pred_cosine < 0:
                pred_cosine = 0
            if pred_cosine > 5:
                pred_cosine = 5
            if pred_jaccard < 0:
                pred_jaccard = 0
            if pred_jaccard > 5:
                pred_jaccard = 5
            if pred_pearson < 0:
                pred_pearson = 0
            if pred_pearson > 5:
                pred_pearson = 5

            pred_rate_cosine.append(pred_cosine)
            pred_rate_jaccard.append(pred_jaccard)
            pred_rate_pearson.append(pred_pearson)

        rmse_cosine.append(sqrt(mean_squared_error(true_rate,pred_rate_cosine)))
        rmse_jaccard.append(sqrt(mean_squared_error(true_rate,pred_rate_jaccard)))
        rmse_pearson.append(sqrt(mean_squared_error(true_rate,pred_rate_pearson)))

      
    rmse_cosine = sum(rmse_cosine)/float(len(rmse_cosine))
    rmse_pearson = sum(rmse_pearson)/float(len(rmse_pearson))
    rmse_jaccard = sum(rmse_jaccard)/float(len(rmse_jaccard))

    print("rmse_cosine = " + str(rmse_cosine) + "\t" + "rmse_jaccard = " + str(rmse_jaccard) + "\t" + "rmse_pearson = " + str(rmse_pearson))

    rmse = [rmse_cosine , rmse_jaccard , rmse_pearson]

    req_sim = rmse.index(min(rmse))

    if req_sim == 0:
        print('sim_mat_user = cosine')
        sim_mat_user = sim_user_cosine
    if req_sim == 1:
        print('sim_mat_user = jaccard')
        sim_mat_user = sim_user_jaccard
    if req_sim == 2:
        print('sim_mat_user = pearson')
        sim_mat_user = sim_user_pearson

    return Mat , sim_mat_user


In [20]:
def predictRating(recommend_data):
    data , sim_user = crossValidation(recommend_data)
    f = open("toBeRated.csv","r")
    toBeRelated =np.zeros((items),dtype=np.float64)
    user_id = 0
    ct=0
    for row in f:
        if ct>0:
            x = int(row)
            toBeRelated[x-1] = 1.0
        ct = ct+1
    f.close()
   
    user_similarity = np.zeros((users),dtype=np.float64)

    for user1 in range(users):
        if np.count_nonzero(data[user1]):
            if req_sim == 0:
                user_similarity[user1] = 1.0 - scipy.spatial.distance.cosine(toBeRelated , data[user1])
            else:
                if req_sim == 1:

                    user_similarity[user1] = 1.0 - scipy.spatial.distance.jaccard(toBeRelated,data[user1])
                else:
                    try :
                        if not math.isnan(scipy.stats.prearson(toBeRelated,data[user1])[0]):
                            user_similarity[user1] = scipy.stats.pearson(toBeRelated,data[user1])[0]
                        else:
                            user_similarity[user1] = 0
                    except:
                        user_similarity[user1]  = 0
   
    print('user_similarity')
    print(user_similarity)
    print('similar users ::: ')
    l = user_similarity.argsort()[-10:][::-1]
    print(l)
    
    f = open("worked.csv","r")
    job = []
    for row in f:
        a = row.split('\t')
        a = a[0].split(',')
        a[len(a)-1] = a[len(a)-1][:-2]
        job.append(a)
    new_job=[]
    for j in job:
        x = []
        for i in j:
            if i != ' ':
                x.append(i)
        new_job.append(x)
    f.close()
    job=[]
    for i in range(len(new_job)):
        x =[]
        for j in range(len(new_job[i])):
            if new_job[i][j] != '':
                x.append(new_job[i][j])
        job.append(x)
    
    vacancy = []
    for i in l:
        for j in job[i]:
            if not j in vacancy:
                vacancy.append(j)

    
    
    v = []
    for i in range(len(vacancy)):
        if len(vacancy[i])==4:
            v.append(vacancy[i])
    print("Job offers :: ")
    print(v)
    return v

In [25]:
def recommend() :
    data = readingFile("aman_rating.csv")
    predictRating(data)

In [26]:
recommend()

rmse_cosine = 0.6597956312954906	rmse_jaccard = 0.6597956312954906	rmse_pearson = 2.0
sim_mat_user = cosine
user_similarity
[0.47140452 0.         0.23570226 0.23570226 0.         0.
 0.         0.23570226 0.23570226 0.         0.         0.
 0.         0.         0.         0.         0.23570226 0.
 0.23570226 0.33333333]
similar users ::: 
[ 0 19  8  7  2  3 18 16  1  4]
Job offers :: 
['1610', '1614', '1615', '1619', '1620', '1621', '1626', '1631', '1637', '1639', '1640', '1612', '1613', '1622', '1635', '1611', '1632', '1625', '1627', '1628', '1630', '1638', '1634', '1636', '1624', '1617', '1618', '1623', '1616', '1633', '1629']
